In [1]:
import datetime as dt

import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import seaborn as sns
import plotly.express as px

In [2]:
data = pd.read_csv('final_data.csv')
print(data.shape)
data.head(3)

(105, 299)


,name,add,real_add,행정동,price,총층수,1층포함여부,평수(㎡),대관단위(시간),총평수,...,편의점 건수비율_3분기,학원 건수비율_3분기,한식 건수비율_3분기,할인점/슈퍼마켓 건수비율_3분기,화장품 건수비율_3분기,골목_컨셉,갤러리_컨셉,음악_컨셉,예술_컨셉,한옥 개수
0,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 20,성수1가2동,800000.0000,1.0000,1.0000,48.0000,23.0000,48.0000,...,0.0036,0.0012,0.0059,0.0020,0.0061,2.0000,0.0000,0.0000,0.0000,0.0000
1,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 26 1층,성수1가2동,900000.0000,1.0000,1.0000,59.4000,23.0000,59.4000,...,0.0036,0.0012,0.0059,0.0020,0.0061,2.0000,0.0000,0.0000,0.0000,0.0000
2,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구,서울 성동구 서울숲4길 15-1 통건물,성수1가2동,6000000.0000,6.0000,0.0000,697.0000,23.0000,4182.0000,...,0.0036,0.0012,0.0059,0.0020,0.0061,2.0000,0.0000,0.0000,0.0000,0.0000


In [4]:
## 기본 칼럼 

main_col = ['name', 'real_add', '자치구 행정동','price', '총층수', '1층포함여부', '평수(㎡)', '총평수', '수용인원수', 
      '시간당대관료', '평당가격', '공간크기','역번호','랜드마크','카페','버스정류장','도로 등 비율(%)']
data[main_col].head()

,name,real_add,자치구 행정동,price,총층수,1층포함여부,평수(㎡),총평수,수용인원수,시간당대관료,평당가격,공간크기,역번호,랜드마크,카페,버스정류장,도로 등 비율(%)
0,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구 서울숲4길 20,성동구 성수1가2동,800000.0000,1.0000,1.0000,48.0000,48.0000,16.0000,34782.6087,724.6377,소,1.0000,5.0000,158.0000,12.0000,17.3970
1,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구 서울숲4길 26 1층,성동구 성수1가2동,900000.0000,1.0000,1.0000,59.4000,59.4000,20.0000,39130.4348,658.7615,소,1.0000,5.0000,158.0000,12.0000,17.3970
2,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구 서울숲4길 15-1 통건물,성동구 성수1가2동,6000000.0000,6.0000,0.0000,697.0000,4182.0000,1394.0000,260869.5652,62.3791,대,1.0000,5.0000,158.0000,12.0000,17.3970
3,서울숲 2.7,서울 성동구 서울숲2길 27,성동구 성수1가2동,4000000.0000,1.0000,1.0000,60.0000,60.0000,20.0000,173913.0435,2898.5507,소,1.0000,5.0000,158.0000,12.0000,17.3970
4,성수 픽닷 옆 매장 팝업스토어,서울 성동구 서울숲4길 23-1 102호,성동구 성수1가2동,1000000.0000,1.0000,1.0000,26.4000,26.4000,9.0000,43478.2609,1646.9038,소,1.0000,5.0000,158.0000,12.0000,17.3970


In [5]:
def Select_period(start, end):## 기간 선택
    start_month = pd.to_datetime(start).month
    end_month = pd.to_datetime(end).month
    month_list = [start_month, end_month]

    start_quater = pd.to_datetime(start).quarter
    end_quater = pd.to_datetime(end).quarter
    quater_list = [start_quater, end_quater]

    if month_list[0] == month_list[-1]:
        month_list = [month_list[0]]
    if quater_list[0] == quater_list[-1]:
        quater_list = [quater_list[0]]

    return month_list, quater_list

In [6]:
def Select_month(month_list):## 생활인구, 축제 선택 (기간)
    month_col = []

    for month in month_list:    
        month_col.append(f'2030 생활인구 {month}월')
        month_col.append(f'2023년{month}월행사')
    
    return month_col

In [7]:
def Select_concept(concept_input): ## 컨셉 선택
    concept_col = []
    
    concept_dict = {'없음' : [],
                    '자연친화': ['자연 비율(%)'],
                   '인더스트리얼': ['공장용지 등 비율(%)'],
                   '전통': ['한옥 개수'],
                   '문화/예술': ['예술_컨셉','갤러리_컨셉','음악_컨셉'],
                   '골목' : ['골목_컨셉']}
    
    for key in concept_input:
        concept_col += concept_dict[key]
    return concept_col

In [23]:
def Select_category(category_input): ## 업종 선택
    category_col = []

    for quater in quater_list:
        category_dict = {'음식' : [f'기타요식 금액비율_{quater}분기',f'음/식료품 금액비율_{quater}분기', f'일식/중식/양식 금액비율_{quater}분기',f'한식 금액비율_{quater}분기',
                            f'기타요식 건수비율_{quater}분기',f'음/식료품 건수비율_{quater}분기', f'일식/중식/양식 건수비율_{quater}분기',f'한식 건수비율_{quater}분기'],
     '디저트' : [f'제과/커피/패스트푸드 금액비율_{quater}분기',
             f'제과/커피/패스트푸드 건수비율_{quater}분기'],
     '가전/가구' : [f'가전/가구 금액비율_{quater}분기',f'인테리어 금액비율_{quater}분기',
              f'가전/가구 건수비율_{quater}분기' ,f'인테리어 건수비율_{quater}분기'],
     '교육' :  [f'교육용품 금액비율_{quater}분기',f'학원 금액비율_{quater}분기', f'유아교육 금액비율_{quater}분기',
             f'교육용품 건수비율_{quater}분기',f'학원 건수비율_{quater}분기', f'유아교육 건수비율_{quater}분기'], 
     '자동차' : [f'자동차서비스/용품 금액비율_{quater}분기', f'자동차판매 금액비율_{quater}분기',f'주유 금액비율_{quater}분기',
             f'자동차서비스/용품 건수비율_{quater}분기', f'자동차판매 건수비율_{quater}분기',f'주유 건수비율_{quater}분기'],
     '여행숙박' : [f'숙박 금액비율_{quater}분기',f'여행 금액비율_{quater}분기',
              f'숙박 건수비율_{quater}분기',f'여행 건수비율_{quater}분기'], 
     '스포츠' : [f'스포츠/문화/레저 금액비율_{quater}분기', f'스포츠/문화/레저용품 금액비율_{quater}분기',
             f'스포츠/문화/레저 건수비율_{quater}분기', f'스포츠/문화/레저용품 건수비율_{quater}분기'], 
     '미용' : [f'미용서비스 금액비율_{quater}분기', f'화장품 금액비율_{quater}분기',
            f'미용서비스 건수비율_{quater}분기', f'화장품 건수비율_{quater}분기'], 
     '패션' : [f'의복/의류 금액비율_{quater}분기', f'패션/잡화 금액비율_{quater}분기',
            f'의복/의류 건수비율_{quater}분기', f'패션/잡화 건수비율_{quater}분기'], 
     '생활' : [f'편의점 금액비율_{quater}분기', f'할인점/슈퍼마켓 금액비율_{quater}분기',
            f'편의점 건수비율_{quater}분기', f'할인점/슈퍼마켓 건수비율_{quater}분기'],
     '유흥' : [f'유흥 금액비율_{quater}분기',
            f'유흥 건수비율_{quater}분기'], 
     '럭셔리' : [f'백화점 금액비율_{quater}분기',
             f'백화점 건수비율_{quater}분기']}

        for key in category_input:
            category_col += category_dict[key]
    return category_col

In [24]:
start, end = '2024-05-15', '2024-05-30' ## 스트림릿으로 입력 받음 (시작일,종료일)
concept_input = ['자연친화'] ## 스트림릿으로 입력 받음 (컨셉)
category_input = ['음식','디저트','가전/가구'] ## 스트림릿으로 입력 받음 (업종)

month_list, quater_list = Select_period(start, end)
month_col = Select_month(month_list)
concept_col = Select_concept(concept_input)
category_col = Select_category(category_input)

In [25]:
## 최종 칼럼 선택 (메인 칼럼 + 기간 칼럼 + 컨셉 칼럼 + 업종 칼럼)

chosen_col = main_col + month_col + concept_col + category_col
print(chosen_col,end='')

## 분석할 데이터 (최종 칼럼 선택)
chosen_data = data[chosen_col]
display(chosen_data.head(3))

['name', 'real_add', '자치구 행정동', 'price', '총층수', '1층포함여부', '평수(㎡)', '총평수', '수용인원수', '시간당대관료', '평당가격', '공간크기', '역번호', '랜드마크', '카페', '버스정류장', '도로 등 비율(%)', '2030 생활인구 5월', '2023년5월행사', '자연 비율(%)', '기타요식 금액비율_2분기', '음/식료품 금액비율_2분기', '일식/중식/양식 금액비율_2분기', '한식 금액비율_2분기', '기타요식 건수비율_2분기', '음/식료품 건수비율_2분기', '일식/중식/양식 건수비율_2분기', '한식 건수비율_2분기', '제과/커피/패스트푸드 금액비율_2분기', '제과/커피/패스트푸드 건수비율_2분기', '가전/가구 금액비율_2분기', '인테리어 금액비율_2분기', '가전/가구 건수비율_2분기', '인테리어 건수비율_2분기']

,name,real_add,자치구 행정동,price,총층수,1층포함여부,평수(㎡),총평수,수용인원수,시간당대관료,...,기타요식 건수비율_2분기,음/식료품 건수비율_2분기,일식/중식/양식 건수비율_2분기,한식 건수비율_2분기,제과/커피/패스트푸드 금액비율_2분기,제과/커피/패스트푸드 건수비율_2분기,가전/가구 금액비율_2분기,인테리어 금액비율_2분기,가전/가구 건수비율_2분기,인테리어 건수비율_2분기
0,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구 서울숲4길 20,성동구 성수1가2동,800000.0000,1.0000,1.0000,48.0000,48.0000,16.0000,34782.6087,...,0.0086,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088
1,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구 서울숲4길 26 1층,성동구 성수1가2동,900000.0000,1.0000,1.0000,59.4000,59.4000,20.0000,39130.4348,...,0.0086,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088
2,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구 서울숲4길 15-1 통건물,성동구 성수1가2동,6000000.0000,6.0000,0.0000,697.0000,4182.0000,1394.0000,260869.5652,...,0.0086,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088


---
## 00. K-Means 모델링

In [26]:
def model_processing(chosen_data):
    obj_col = list(chosen_data.select_dtypes('object').columns)
    num_data = chosen_data.drop(obj_col,axis=1) # 스케일링을 위해 범주형 제외
    
    ## 스케일링 : 칼럼이 선택되는 것에 따라 스케일링이 따로 진행됨
    from sklearn.preprocessing import MinMaxScaler
    Scaler = MinMaxScaler()
    Scaled = Scaler.fit_transform(num_data)
    return Scaled

In [27]:
Scaled = model_processing(chosen_data)

In [28]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters= 3, init='k-means++', random_state=2024)
k_means.fit_predict(Scaled)
chosen_data['cluster'] = k_means.labels_
display(chosen_data.head())


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_18500\3269856402.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chosen_data['cluster'] = k_means.labels_


,name,real_add,자치구 행정동,price,총층수,1층포함여부,평수(㎡),총평수,수용인원수,시간당대관료,...,음/식료품 건수비율_2분기,일식/중식/양식 건수비율_2분기,한식 건수비율_2분기,제과/커피/패스트푸드 금액비율_2분기,제과/커피/패스트푸드 건수비율_2분기,가전/가구 금액비율_2분기,인테리어 금액비율_2분기,가전/가구 건수비율_2분기,인테리어 건수비율_2분기,cluster
0,성수 팝업 서울숲 위팝 1st by SHARE.IT,서울 성동구 서울숲4길 20,성동구 성수1가2동,800000.0000,1.0000,1.0000,48.0000,48.0000,16.0000,34782.6087,...,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088,1
1,성수 팝업 서울숲 위팝 2nd by SHARE.IT,서울 성동구 서울숲4길 26 1층,성동구 성수1가2동,900000.0000,1.0000,1.0000,59.4000,59.4000,20.0000,39130.4348,...,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088,1
2,성수 서울숲 팝업 건물 통대관 by SHARE.IT,서울 성동구 서울숲4길 15-1 통건물,성동구 성수1가2동,6000000.0000,6.0000,0.0000,697.0000,4182.0000,1394.0000,260869.5652,...,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088,1
3,서울숲 2.7,서울 성동구 서울숲2길 27,성동구 성수1가2동,4000000.0000,1.0000,1.0000,60.0000,60.0000,20.0000,173913.0435,...,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088,1
4,성수 픽닷 옆 매장 팝업스토어,서울 성동구 서울숲4길 23-1 102호,성동구 성수1가2동,1000000.0000,1.0000,1.0000,26.4000,26.4000,9.0000,43478.2609,...,0.0018,0.0099,0.0066,0.0176,0.0103,0.0024,0.0038,0.0013,0.0088,1


In [29]:
cluster_centroid = chosen_data.groupby(['cluster'],as_index=False).mean()
cluster_centroid

C:\Users\user\AppData\Local\Temp\ipykernel_18500\3921277859.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cluster_centroid = chosen_data.groupby(['cluster'],as_index=False).mean()


,cluster,price,총층수,1층포함여부,평수(㎡),총평수,수용인원수,시간당대관료,평당가격,역번호,...,기타요식 건수비율_2분기,음/식료품 건수비율_2분기,일식/중식/양식 건수비율_2분기,한식 건수비율_2분기,제과/커피/패스트푸드 금액비율_2분기,제과/커피/패스트푸드 건수비율_2분기,가전/가구 금액비율_2분기,인테리어 금액비율_2분기,가전/가구 건수비율_2분기,인테리어 건수비율_2분기
0,0,2611666.6667,1.6000,0.7333,258.9585,546.5585,182.1667,410484.0982,3431.4241,1.6000,...,0.0115,0.0053,0.0169,0.0088,0.0104,0.0086,0.0194,0.0075,0.0022,0.0076
1,1,1782471.4286,2.4857,0.8714,151.2258,461.0362,153.7857,130145.5689,798.4413,1.0000,...,0.0071,0.0016,0.0084,0.0052,0.0126,0.0078,0.0021,0.0036,0.0010,0.0069
2,2,704000.0000,1.2000,1.4000,2899.1000,2995.1000,998.4000,380666.6667,906.9560,5.0000,...,0.0307,0.0085,0.0340,0.0317,0.0253,0.0256,0.1067,0.0022,0.0233,0.0062


In [30]:
## 사용자에게 보여줄 데이터

cluster_0 = chosen_data[chosen_data['cluster'] == 0][['name','real_add','자치구 행정동','price','총평수','시간당대관료','평당가격','공간크기']]
cluster_1 = chosen_data[chosen_data['cluster'] == 1][['name','real_add','자치구 행정동','price','총평수','시간당대관료','평당가격','공간크기']]
cluster_2 = chosen_data[chosen_data['cluster'] == 2][['name','real_add','자치구 행정동','price','총평수','시간당대관료','평당가격','공간크기']]

In [31]:
cluster_0['자치구 행정동'].unique(),cluster_1['자치구 행정동'].unique(),cluster_2['자치구 행정동'].unique()

(array(['강남구 압구정동', '강남구 신사동', '중구 회현동', '중구 명동', '광진구 화양동'], dtype=object),
 array(['성동구 성수1가2동', '성동구 성수1가1동', '서초구 양재1동', '서초구 서초3동', '강남구 대치4동',
        '종로구 가회동', '중구 신당동'], dtype=object),
 array(['강남구 역삼1동'], dtype=object))

In [1]:
# ## 머신러닝 모델 저장 : 편의를 위해 특정 조건 이름으로 저장
# import joblib

# joblib.dump(k_means, f'K-means_clustering_{start,end},{concept_input},{category_input}.pkl')
# chosen_data.to_csv(f'cluster_data_{start,end},{concept_input},{category_input}.csv',index=False)

# print(f'K-means_clustering_{start,end},{concept_input},{category_input}.pkl')
# print(f'cluster_data_{start,end},{concept_input},{category_input}.csv')

## END OF SCRIPT